## Notebook walking through updating the text file - this could be automated, but keeping it as a notebook allows for an interactive process to QA/QC the descriptions and make sure they were scraped correctly. 

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('ref_files/product_info.csv')
data.head()

,Name,Description,Material
0,The Waylons in Chestnut,"Built by our buddies at VALLON, The Waylons is...","The Waylons’ frame is made with lightweight, d..."
1,The Sashiko Denim Repair Kit,"Sure, you could hand off your well-worn denim ...",The Sashiko Denim Repair Kit includes an assor...
2,The Camp Candle in Shoreline,The Camp Candle was poured by hand in small ba...,"Made with natural, renewable soy wax. Soy wax ..."
3,The Cotton Hemp Tee in Charcoal Open Road,This exclusive edition of The Cotton Hemp Tee ...,Organic cotton offers all the benefits of the ...
4,The Cotton Hemp Tee in Navy Give to Get,Regenerative agriculture is all about reciproc...,We love organic cotton because it offers all o...


In [3]:
data.shape

(1209, 3)

In [12]:
# different locations to scrape product data...

#other_url = 'https://www.taylorstitch.com/collections/2022-summer-sale-archive-bring-back?sorted=best-selling-sales-count'
shirts_url = 'https://www.taylorstitch.com/collections/mens-shirts'
#bottoms_url = 'https://www.taylorstitch.com/collections/mens-bottoms'
#knits_url = 'https://www.taylorstitch.com/collections/mens-knits'
#outerwear_url = 'https://www.taylorstitch.com/collections/mens-outerwear'
#last_call_url = 'https://www.taylorstitch.com/collections/mens-last-call'

base_url = 'https://www.taylorstitch.com'
new_arrivals = 'https://www.taylorstitch.com/collections/mens-new-arrivals'
url_list = [new_arrivals]

In [14]:
product_urls = [] # list of all product pages to get later

for url in url_list:
    site = requests.get(url)

    soup = BeautifulSoup(site.text, 'html.parser')
    #print(soup)
    products = soup.find_all('ul',{'class':'product matrix sitewide-sale'})
    print(products)
    products = products[0].find_all('a',href=True) # a tags hold products here

    for product in products:
        product_urls.append(base_url+product['href'])
        
# look at each product, pull relevent info.
# store all info in lists, which will be converted to pandas df later
product_title = []
product_description = []
product_material = []


for product in product_urls:
    product_page = requests.get(product)
    product_soup = BeautifulSoup(product_page.text, 'html.parser')
    
    title_info = product_soup.find('h1')['data-title']
    if data['Name'].str.contains(title_info).any():
      continue
    else: print('adding',title_info)

    description_info = product_soup.find_all('div',
                                             {'id':'collapsible-description'})
    material_info = product_soup.find_all('div',
                                             {'id':'collapsible-material'})

    try: 
        description = description_info[0].find('p').text
        material = material_info[0].find('p').text
    
    except:
        print('could not add info for',title_info) 
        continue
        
    product_title.append(title_info)
    product_description.append(description)
    product_material.append(material)


<!DOCTYPE html>

<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head><meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0, shrink-to-fit=no" name="viewport"/>
<link href="https://www.taylorstitch.com/collections/mens-new-arrivals" rel="canonical"/><link href="https://www.taylorstitch.com/sitemap.xml" rel="sitemap" title="Sitemap" type="application/xml"/>
<link href="https://www.taylorstitch.com/blogs/archive.atom" rel="alternate" title="Atom" type="application/atom+xml"/>
<link href="https://www.taylorstitch.com/collections/all.atom" rel="alternate" title="Atom" type="application/atom+xml"/><link href="https://cdn.shopify.com/s/files/1/0070/1922/t/128/assets/favicon.ico?v=144679804329704246851660868088" id="favicon" rel="shortcut icon" type="image/x-icon"/>
<link href="https://c

adding The Crawford Sweater in Charcoal
adding The Crawford Sweater in Rust
adding The Jack in Terrace Plaid
adding The Jack in Academy Plaid
adding The Democratic All Day Pant in Walnut Cord
adding The Democratic All Day Pant in Khaki Cord
adding The Slim All Day Pant in Walnut Cord
adding The Democratic All Day Pant in Cypress Cord
adding The Slim All Day Pant in Khaki Cord
adding The Slim All Day Pant in Cypress Cord
adding The Lodge Scarf in Coffee
adding The Lodge Beanie in Rust
adding The Able Vest in Quilted Army
adding The Utility Shirt in Walnut Double Cloth
adding The Utility Shirt in Stone Double Cloth
adding The Utility Shirt in Russet Double Cloth
adding The Everett Sweater in Navy Birdseye
adding The Lodge Beanie in Coffee
adding The Lodge Scarf in Camel
adding The Lodge Beanie in Camel
adding The Rugby Shirt in Dark Navy
adding The Lodge Scarf in Rust
adding The Fillmore Crewneck in Copper Terry
adding The Fillmore Crewneck in Cypress Terry
adding The Fillmore Hoodie in 

In [ ]:
all_info = pd.DataFrame(list(zip(product_title, 
                                 product_description,
                                 product_material)),
               columns =['Name', 'Description', 'Material'])

In [22]:
all_info = all_info.drop_duplicates()

In [25]:
all_info.shape

(16, 3)

In [10]:
# print each one out to inspect the description 
for item in all_info['Description']:
  print(item,'\n')

Our California shirt embodies the laid back, industrious, earth conscious vibes of our beloved home state, so it’s no surprise that the design is close to our hearts. What we didn’t expect was how much our TS family would love it too. It seems the lightweight feel, tailored silhouette and rugged double-needle felled construction make The California a no-nonsense, no-brainer for folks from all fifty states and beyond. This retro-inspired Trolley Plaid iteration is the perfect pair for everything from slacks, to chinos to your favorite pair of battered blue jeans, so you can take some California spirit with you no matter the occasion. 

Our California shirt embodies the laid back, industrious, earth conscious vibes of our beloved home state, so it’s no surprise that the design is close to our hearts. What we didn’t expect was how much our TS family would love it too. It seems the lightweight feel, tailored silhouette and rugged double-needle felled construction make The California a no-n

In [26]:
# add the new descriptions to the existing file
data = pd.concat([data,all_info])

In [27]:
# look at new size
data.shape

(1209, 3)

In [29]:
# and finally save. 
data.to_csv('ref_files/product_info.csv',index=False,header=True)